In [ ]:
import json, time
from IPython.display import clear_output
from STservo import ST3215, Calibrator 

import socket
import json 

HOST = '0.0.0.0'   # 监听所有网卡
PORT = 5000        # 自定义端口
servo_arm = ST3215(port='/dev/ttyACM0')
def main():
    # 创建 TCP 套接字
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((HOST, PORT))
    server.listen(5)
    print(f"Server listening on {HOST}:{PORT}")

    while True:
        conn, addr = server.accept()
        print(f"Connected by {addr}")

        # 通信循环
        count = 0
        ts = time.time()
        while True:
            count += 1
            length = conn.recv(2)
            if length==b'\x00\x00':
                print(f'{count / (time.time() - ts)}fps')
                break
            data = conn.recv(int.from_bytes(length, byteorder='big'))
            clear_output()#wait=True 
            conf_arm = json.loads(data)
            #print(conf_arm)
            id_arr   = []
            posi_arr = []
            velo_arr = []
            for _id, conf in conf_arm.items():
                id_arr.append(int(_id))
                posi_arr.append(conf['posi'])
                velo_arr.append(conf['velo'])
            servo_arm.move2Posi(id_arr=id_arr, posi_arr=posi_arr, velo_arr=velo_arr)
            conn.sendall(b'\xff\xff')   # 回显数据

        conn.close()
        print(f"Connection closed: {addr}")

if __name__ == "__main__":
    main()